<a href="https://colab.research.google.com/github/shravan207/datascience/blob/master/airbnb_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
##this assignment is contributed to by Matthew Zimmer
## Shravan Seshadri 
## Jiuzhou Zhao
##no James
## G16

import pandas as pd
pd.set_option('display.max_columns', 108)
import numpy as np
from matplotlib import pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import os
import re
from numpy import *

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


 Our initial steps once we find a dataset would be to read in the data and focus on preparing the data in order to use in our model. To do so, we would have to clean our data according to how we need it to enter in our model

## DATA CLEANING, PRE-PROCESSING & HANDLING MISSING VALUES

1) First we read in our data in csv format into our data frame and use the seperator to get it in a tabular format. It is a good choice to print out the data frame using the .head() function to get a look at what we are dealing with.

In [0]:
listings = pd.read_csv("airbnb-seattle-listings-train.csv", sep ="\t")
listings.head()

FileNotFoundError: ignored

2) An important step in Feature engineering is to investigate how many 'features' (aka columns) we have in our dataset. It mainly tells us how many features we have in our dataset and based on this information we can decide which features will be helpful in our model and which we might not be able to use.

3) We come to our next important step which is deciding our missing value threshold. What do I mean by this?
Well, our dataset will present with missing values which mean that these values are either not recorded properly or are not available for us to use. In feature engineering, we tend to not use features with too many missing values as that would affect our prediction from the model. Therefore, we must decide a threshold of missing values we would like our features to have and if a feature crosses that threshold we should consider dropping it from our dataframe. 

3.1) How do we choose this threshold value? Usually with smaller datasets we would like to keep the threshold value smaller so that we dont lose too much data in missing values which would eventually affect our model. We would like to keep it under 10% or so generally, but in case of this dataset, where we have a lot of values, we can consider increasing this threshold. We can also try a trial and error method, where we try a particular threshold value and see how many features we are losing. If it is too much, we can consider increasing this threshold. 

### Investigating features using a missing value threshold of 20%

We can see how many missing values are there in each column and work towards identifying features that fit in our threshold. However, instead of already taking out features that we need and storing them in a different Data Frame, we will do that in later steps. Usually, we would try and eliminate the features which dont cross our threshold immediately but for our own ease we would'nt do that here. 

In [0]:
#print(listings.columns)
print("MISSING VALUES BY COLUMN:")
print(listings.isnull().sum())



MISSING VALUES BY COLUMN:
id                                                 0
listing_url                                        0
scrape_id                                          0
last_scraped                                       0
name                                               1
                                                ... 
calculated_host_listings_count                     0
calculated_host_listings_count_entire_homes        0
calculated_host_listings_count_private_rooms       0
calculated_host_listings_count_shared_rooms        0
reviews_per_month                               1087
Length: 106, dtype: int64


In [0]:
missing = listings.isnull().sum() / len(listings)
ten_percent = listings.columns[missing < 0.20]
print("FEATURES WITHIN OUR MISSING VALUE THRESHOLD:")
print(ten_percent)

FEATURES WITHIN OUR MISSING VALUE THRESHOLD:
Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location',
       'host_response_time', 'host_response_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'street', 'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities',
       'security_deposit', 'cleaning_fee', 'guests_included', 'extra_people',
       'minimum_nights',

### Cleaning 'price', our independent variable (variable to predict)

Price is a string variable, and in order to use it in our data we need to convert it into a float(numerical) variable
using various available functions in python. We declare a clean price function and apply it to the price feature, our variable to predict and use the logarithmic value of price. (There are several reasons to log our variables in regression, one of the most important being the influence of outliers in our data. Since this dataset is from airbnb listings we, had a couple of outliers and needed a logarithmic scale for our variables to reduce this particular influence of outliers in our regression model.)

In [0]:
print(listings['price'])

0       $296.00
1        $48.00
2        $62.00
3        $99.00
4       $165.00
         ...   
7535     $80.00
7536    $137.00
7537     $35.00
7538     $35.00
7539    $120.00
Name: price, Length: 7540, dtype: object


As we can see, we have to clean the price variable which is a string with the dollar sign. We need to convert into a float variable and take the loagrithmic value

In [0]:
def clean_price(x):
    if isinstance(x, str):
        return(x.replace('$', '').replace(',', ''))
    return(x)
listings['price'] = listings['price'].apply(clean_price).astype('float')
listings=listings[listings['price'] != 0.0]
listings['price'] = where(listings['price'] != 0, np.log(listings['price']), 0)
#listings['price'] = np.log(listings.price)
print(listings['price'])

0       5.690359
1       3.871201
2       4.127134
3       4.595120
4       5.105945
          ...   
7535    4.382027
7536    4.919981
7537    3.555348
7538    3.555348
7539    4.787492
Name: price, Length: 7539, dtype: float64


## GETTING TO THE CRUX OF FEATURE ENGINEERING:
The next two code cells focus on how we engineer features to suit our model. Essentially using existing features and applying your own algorithm to make it more efficient to use in our model.
Below, we are trying to use the variable 'room_type' to find out two things: The average price for each type of room and the maximum of those average prices for each kind of room. (REMEMBER THESE ARE LOGARITHMIC CALCULATIONS, SO DO NOT WORRY IF YOU SEE SMALL VALUES )

In [0]:
roomcheck = listings[['price', 'room_type']].copy()
print(roomcheck.head(10))


      price        room_type
0  5.690359  Entire home/apt
1  3.871201  Entire home/apt
2  4.127134     Private room
3  4.595120     Private room
4  5.105945  Entire home/apt
5  4.828314  Entire home/apt
6  4.787492  Entire home/apt
7  4.828314  Entire home/apt
8  5.700444  Entire home/apt
9  3.688879     Private room


In [0]:
avg_prices = roomcheck.groupby('room_type')['price'].mean()
max_price = avg_prices.max()

print(avg_prices)
print("Maximum of average price:", max_price)

room_type
Entire home/apt    5.040452
Hotel room         5.177841
Private room       4.210111
Shared room        3.644351
Name: price, dtype: float64
Maximum of average price: 5.177841326260519


### Quantifying our categorical variable room type into a numerical score:
Here, we are using the categorical variable room type to calculate our room type score. Essentially, we are using the two things we calculated above to build a numerical percentage score we can assign to each room type to use in our model. We first divide the average price of that room type by the maximum of average prices and turn it into a numerical percentage which we assign to that room type

In [0]:
score_list = []
for roomprice in avg_prices:
    score = (roomprice/max_price) * 100
    final_rounded_score = np.round(score, decimals = 2)
    score_list.append(final_rounded_score)  
#print(score_list)   
#score_list=score_list[score_list['price'] != 0]
tomerge = pd.DataFrame({'room_type': ['Entire home/apt','Hotel room','Private room','Shared room'],
                        'room_type_scores':score_list})
tomerge.head(4)

,room_type,room_type_scores
0,Entire home/apt,97.35
1,Hotel room,100.00
2,Private room,81.31
3,Shared room,70.38


### We add our newly engineered feature to our data frame:
We merge the data frame of room type scores with our original data frame. By merging it on room type we use our newly calculated percentage score for each room type, which will show on the data frame along with the room type

In [0]:
final_listings = pd.merge(listings,tomerge,on ='room_type')
final_listings.head(5)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,room_type_scores
0,2318,https://www.airbnb.com/rooms/2318,20190922030624,2019-09-22,Casa Madrona - Urban Oasis 1 block from the park!,"Gorgeous, architect remodeled, 1917 Dutch Colo...","Casa Madrona is a gorgeous, architect remodele...","Gorgeous, architect remodeled, 1917 Dutch Colo...",none,Madrona is a hidden gem of a neighborhood. It ...,"We adhere to a 10pm -9am quiet hour schedule, ...",NaN,Guests can access any part of the house.,We are a family who live next door and are ava...,NaN,NaN,NaN,https://a0.muscache.com/im/pictures/02973ad3-a...,NaN,2536,https://www.airbnb.com/users/show/2536,Megan,2008-08-26,"Seattle, Washington, United States",I welcome guests from all walks of life and ev...,within an hour,100%,NaN,t,https://a0.muscache.com/im/pictures/user/016a1...,https://a0.muscache.com/im/pictures/user/016a1...,Minor,2.0,2.0,"['email', 'phone', 'reviews', 'jumio', 'offlin...",t,f,"Seattle, WA, United States",Madrona,Madrona,Central Area,Seattle,WA,98122,Seattle,"Seattle, WA",US,United States,47.61082,-122.29082,t,House,Entire home/apt,9,2.5,4.0,4.0,Real Bed,"{Internet,Wifi,Kitchen,""""Free parking on premi...",NaN,5.690359,NaN,NaN,$500.00,$250.00,8,$25.00,30,1000,30,30,1000,1000,30.0,1000.0,5 days ago,t,25,55,84,84,2019-09-22,28,8,2008-09-15,2019-08-30,100.0,10.0,10.0,10.0,10.0,10.0,10.0,t,NaN,"{WASHINGTON,"""" Seattle"""","""" WA""""}",f,f,strict_14_with_grace_period,f,f,2,2,0,0,0.21,97.35
1,5682,https://www.airbnb.com/rooms/5682,20190922030624,2019-09-22,"Cozy Studio, min. to downtown -WiFi",The Cozy Studio is a perfect launchpad for you...,"Hello fellow travelers, Save some money and ha...",The Cozy Studio is a perfect launchpad for you...,none,NaN,My personal favorite places to dine are: • Ma...,Bus 120 is right outside and goes into downtow...,PRIVATE ENTRANCE WITH PRIVATE BATH Queen size ...,"I'm more than happy to meet with guests, but u...",**PLEASE READ** NO SMOKING NO VAPING NO MARIJU...,NaN,NaN,https://a0.muscache.com/im/pictures/5ec89f9d-8...,NaN,8993,https://www.airbnb.com/users/show/8993,Maddy,2009-03-03,"Seattle, Washington, United States",Hello my name is Maddy. I enjoy meeting and ho...,NaN,NaN,NaN,t,https://a0.muscache.com/im/users/8993/profile_...,https://a0.muscache.com/im/users/8993/profile_...

### Creating our neighborhood score feature:
Below, we are trying to clean the "neighbourhoods" variable. As it is a categorical variable, we will quantify them by our own algorithm. We find the maximum price among the neighbourhoods in our dataset and let every single value of price be divided by this max price, thus we give each neighbourhood a "score" in order to quantify them.


In [0]:
missing_values = ['n/a', 'na', '--', 'NA', 'NaN']
airbnb=pd.read_csv("airbnb-seattle-listings-train.csv", sep='\t', na_values=missing_values)


In [0]:
airbnb["price"]=[x[1:] for x in airbnb["price"]]
#airbnb=where(airbnb['price'] != 0, np.log(airbnb['price']), 0)


In [0]:
grouped_price=pd.Series(listings.groupby("neighbourhood").price)

In [0]:
neighbour_mean=pd.Series([pd.to_numeric(x[1], errors='coerce').mean() for x in grouped_price], index=[x[0] for x in grouped_price])

In [0]:
neighbour_mean=np.log(neighbour_mean)

In [0]:
sorted_neighbours=neighbour_mean.sort_values(axis=0, ascending=False)

In [0]:
max_price = neighbour_mean.max()
neighbour_scores_database=pd.Series([(x/max_price)*100 for x in neighbour_mean], index=[x[0] for x in grouped_price])

In [0]:
max_price = neighbour_mean.max()
neighbour_scores_database=pd.DataFrame(data={'neighbourhood':[x[0] for x in grouped_price], 'neighbour_scores':[(x/max_price)*100 for x in neighbour_mean]})
neighbour_scores_database

,neighbourhood,neighbour_scores
0,Alki,90.734342
1,Arbor Heights,85.047385
2,Atlantic,87.727372
3,Ballard,89.504423
4,Belltown,93.152102
5,Bitter Lake,84.913822
6,Brighton,85.412378
7,Broadview,84.905077
8,Bryant,86.156110
9,Capitol Hill,89.213590


We now merge the table with cleaned neighbourhoods and other variables to get our final dataframe to choose features for training our model


In [0]:
#we merge the table with cleaned neighbourhoods and other variables

final_table = pd.merge(final_listings,neighbour_scores_database,on ='neighbourhood')
final_table.head(10)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,room_type_scores,neighbour_scores
0,2318,https://www.airbnb.com/rooms/2318,20190922030624,2019-09-22,Casa Madrona - Urban Oasis 1 block from the park!,"Gorgeous, architect remodeled, 1917 Dutch Colo...","Casa Madrona is a gorgeous, architect remodele...","Gorgeous, architect remodeled, 1917 Dutch Colo...",none,Madrona is a hidden gem of a neighborhood. It ...,"We adhere to a 10pm -9am quiet hour schedule, ...",NaN,Guests can access any part of the house.,We are a family who live next door and are ava...,NaN,NaN,NaN,https://a0.muscache.com/im/pictures/02973ad3-a...,NaN,2536,https://www.airbnb.com/users/show/2536,Megan,2008-08-26,"Seattle, Washington, United States",I welcome guests from all walks of life and ev...,within an hour,100%,NaN,t,https://a0.muscache.com/im/pictures/user/016a1...,https://a0.muscache.com/im/pictures/user/016a1...,Minor,2.0,2.0,"['email', 'phone', 'reviews', 'jumio', 'offlin...",t,f,"Seattle, WA, United States",Madrona,Madrona,Central Area,Seattle,WA,98122,Seattle,"Seattle, WA",US,United States,47.61082,-122.29082,t,House,Entire home/apt,9,2.5,4.0,4.0,Real Bed,"{Internet,Wifi,Kitchen,""""Free parking on premi...",NaN,5.690359,NaN,NaN,$500.00,$250.00,8,$25.00,30,1000,30,30,1000,1000,30.0,1000.0,5 days ago,t,25,55,84,84,2019-09-22,28,8,2008-09-15,2019-08-30,100.0,10.0,10.0,10.0,10.0,10.0,10.0,t,NaN,"{WASHINGTON,"""" Seattle"""","""" WA""""}",f,f,strict_14_with_grace_period,f,f,2,2,0,0,0.21,97.35,91.044225
1,639130,https://www.airbnb.com/rooms/639130,20190922030624,2019-09-22,Fabulous home for a family!,NaN,This is the perfect home for a family wanting ...,This is the perfect home for a family wanting ...,none,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://a0.muscache.com/im/pictures/8225314/4d...,NaN,3187723,https://www.airbnb.com/users/show/3187723,Ann,2012-08-07,"Seattle, Washington, United States",Active family living in the heart of Seattle. ...,within a day,100%,NaN,f,https://a0.muscache.com/im/pictures/user/364e2...,https://a0.muscache.com/im/pictures/user/364e2...,Madrona,1.0,1.0,"['email', 'phone', 'facebook', 'reviews', 'jum...",t,t,"Seattle, WA, United States",Madrona,Madrona,Central Area,Seattle,WA,98122,Seattle,"Seattle, WA",US,United States,47.60975,-122.29011,t,House,Entire home/apt,8,3.0,4.0,5.0

In [0]:
#this is the final database we need to use for model building
test_df = final_table[['id','price','room_type_scores','accommodates','bathrooms','bedrooms','guests_included','host_total_listings_count','neighbour_scores']].copy()
# This is the summary table with all relevant information of the variables we want to use for the model
test_df.describe()

,id,price,room_type_scores,accommodates,bathrooms,bedrooms,guests_included,host_total_listings_count,neighbour_scores
count,7.539000e+03,7539.000000,7539.000000,7539.000000,7539.000000,7538.000000,7539.000000,7538.000000,7539.000000
mean,2.097325e+07,4.850110,93.672998,3.670513,1.311513,1.379942,1.995357,125.062483,89.416729
std,1.086357e+07,0.694319,7.163679,2.286798,0.658401,1.021453,1.676347,380.005561,3.689903
min,2.318000e+03,2.302585,70.380000,1.000000,0.000000,0.000000,1.000000,0.000000,81.031188
25%,1.300444e+07,4.442651,97.350000,2.000000,1.000000,1.000000,1.000000,1.000000,87.370620
50%,2.144653e+07,4.787492,97.350000,3.000000,1.000000,1.000000,1.000000,2.000000,88.914744
75%,3.032114e+07,5.247024,97.350000,4.000000,1.500000,2.000000,2.000000,10.000000,91.984466
max,3.879648e+07,8.594154,100.000000,28.000000,16.000000,8.000000,16.000000,1795.000000,100.000000


In [0]:
#kick out null variables
test_df = test_df.dropna(how='any',axis=0)

##  this is the end of problem 2 and problem 1


## here starts problem 3

In [0]:
#problem3 part(a)
# i m choosing neighbor scores and bathrooms

#step 0 configuration
import pandas as pd
import statsmodels.api as sm 
import statsmodels.formula.api as smf 
import numpy as np
from sklearn.model_selection import train_test_split
from math import sqrt
from sklearn.metrics import mean_squared_error
import math
import matplotlib.pyplot as plt 
from scipy import stats
import cmath


#step I: extract neighbors and bathrooms and split the final-table as 0.2/0.8
#test_df=test_df[test_df['price'] != 0.0]
#print(test_df)
#x=test_df[['accommodates', 'neighbour_scores']]
#y=test_df['price'].apply(lambda x: math.log(x))
train, val  = train_test_split(test_df, test_size=0.2, random_state=1)


In [0]:
 
# step II: build model on the tr data and show summary
tr = train[['accommodates',
       'neighbour_scores', 'price']]

te = val[['accommodates',
       'neighbour_scores', 'price']]

model = smf.ols(formula = "price ~ accommodates+ neighbour_scores", data = tr).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.462
Model:                            OLS   Adj. R-squared:                  0.462
Method:                 Least Squares   F-statistic:                     2585.
Date:                Fri, 06 Dec 2019   Prob (F-statistic):               0.00
Time:                        01:19:10   Log-Likelihood:                -4470.9
No. Observations:                6029   AIC:                             8948.
Df Residuals:                    6026   BIC:                             8968.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           -3.4534      0.159    -21.693      0.000      -3.765      -3.141
accommodates         0.1496      0.003     52.588      0.000       0.144       0.155
neighbour_scores     0.0867      0.002     48.818      0.000       0.083       0.090
==============================================================================
Omnibus:                      601.278   Durbin-Watson:                   1.987
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3938.479
Skew:                           0.231   Prob(JB):                         0.00
Kurtosis:                       6.933   Cond. No.                     2.18e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.18e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [0]:
Xte = te[['accommodates',
       'neighbour_scores', 'price']]
#test_dum = pd.get_dummies(Xte)
#test_encoded_for_model = test_dum.reindex(columns = tr[['accommodates', 'neighbour_scores']].columns, 
 #   fill_value=0)
predicted_validation = model.predict(Xte)


In [0]:
# step III: use te data to report rmse
rmse = sqrt(mean_squared_error(Xte["price"], predicted_validation))
#final rmse is 
rmse

0.505107876482341

In [0]:
#problem3 part(b)
#step I: extract neighbors and bathrooms and split the final-table as 0.2/0.8
train, val  = train_test_split(test_df, test_size=0.2, random_state=1)

# step II: build model on the tr data and show summary
model = smf.ols(formula = "price ~ id + room_type_scores + accommodates +  bathrooms + bedrooms + guests_included + host_total_listings_count + neighbour_scores", data = train).fit()

model.summary()



<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.592
Model:                            OLS   Adj. R-squared:                  0.592
Method:                 Least Squares   F-statistic:                     1093.
Date:                Fri, 06 Dec 2019   Prob (F-statistic):               0.00
Time:                        01:19:10   Log-Likelihood:                -3634.0
No. Observations:                6029   AIC:                             7286.
Df Residuals:                    6020   BIC:                             7346.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                    -4.2002      0.162    -25.925      0.000      -4.518      -3.883
id                        -3.105e-09   5.39e-10     -5.757      0.000   -4.16e-09   -2.05e-09
room_type_scores              0.0309      0.001     33.327      0.000       0.029       0.033
accommodates                  0.0361      0.005      7.645      0.000       0.027       0.045
bathrooms                     0.0495      0.011      4.481      0.000       0.028       0.071
bedrooms                      0.1677      0.010     16.907      0.000       0.148       0.187
guests_included               0.0345      0.004      8.012      0.000       0.026       0.043
host_total_listings_count     0.0003    1.7e-05     19.083      0.000       0.000       0.000
neighbour_scores              0.0635      0.002     36.090      0.000       0.060       0.067
==============================================================================
Omnibus:                     1057.745   Durbin-Watson:                   1.942
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             7270.922
Skew:                           0.661   Prob(JB):                         0.00
Kurtosis:                       8.215   Cond. No.                     6.74e+08
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.74e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [0]:
# step III: use te data to report rmse
predicted_validation = model.predict(val)


rmse = sqrt(mean_squared_error(te["price"], predicted_validation))
#final rmse is 
rmse

0.43666717142791706

In [0]:
#problem3 part(c)
#step I: extract neighbors and bathrooms and split the final-table as 0.2/0.8


train, val  = train_test_split(test_df, test_size=0.2, random_state=1)

# step II: build model on the tr data and show summary
tr = train[['accommodates',
       'neighbour_scores', 'bathrooms', 'bedrooms','price']]

te = val[['accommodates',
       'neighbour_scores', 'bathrooms', 'bedrooms','price']]


model = smf.ols(formula = "price ~ accommodates+ neighbour_scores+ bathrooms+ bedrooms", data = tr).fit()


model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.479
Model:                            OLS   Adj. R-squared:                  0.478
Method:                 Least Squares   F-statistic:                     1383.
Date:                Fri, 06 Dec 2019   Prob (F-statistic):               0.00
Time:                        01:19:10   Log-Likelihood:                -4374.4
No. Observations:                6029   AIC:                             8759.
Df Residuals:                    6024   BIC:                             8792.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           -3.8361      0.160    -24.032      0.000      -4.149      -3.523
accommodates         0.1000      0.005     21.517      0.000       0.091       0.109
neighbour_scores     0.0910      0.002     51.256      0.000       0.088       0.094
bathrooms           -0.0234      0.012     -1.921      0.055      -0.047       0.000
bedrooms             0.1516      0.011     13.692      0.000       0.130       0.173
==============================================================================
Omnibus:                      577.196   Durbin-Watson:                   1.973
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3196.130
Skew:                           0.285   Prob(JB):                         0.00
Kurtosis:                       6.521   Cond. No.                     2.22e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.22e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [0]:
# step III: use te data to report rmse
predicted_validation = model.predict(te)


rmse = sqrt(mean_squared_error(te["price"], predicted_validation))
#final rmse is 
rmse

0.4941108044976578

In [0]:
# problem 3 - part 5
#install the below command in your anaconda terminal before you run following codes
#conda install -c conda-forge tabulate
from tabulate import tabulate

rmse_table = [["A",0.51],["B",0.44],["C",0.49]]

print(tabulate(rmse_table, headers=["Model","RMSE"], tablefmt="psql"))
#table here shows 
# the rmse for part a) is 
#0.50510787648234
#the rmse for part b) is
#0.4387605687129302
#the rmse for part c) is 
#0.4941108044976578

+---------+--------+
| Model   |   RMSE |
|---------+--------|
| A       |   0.51 |
| B       |   0.44 |
| C       |   0.49 |
+---------+--------+


## this is the end of problem 3

## here starts problem 4

In [0]:
# problem-4
# part 1
missing_values = ['n/a', 'na', '--', 'NA', 'NaN']
airbnb =pd.read_csv("airbnb-seattle-listings-test.csv", sep='\t', na_values=missing_values)
listings = pd.read_csv("airbnb-seattle-listings-test.csv", sep ="\t")


In [0]:
#part 2
missing = listings.isnull().sum() / len(listings)
ten_percent = listings.columns[missing > 0.20]
def clean_price(x):
    if isinstance(x, str):
        return(x.replace('$', '').replace(',', ''))
    return(x)
listings['price'] = listings['price'].apply(clean_price).astype('float')
listings=listings[listings['price'] != 0.0]
listings['price'] = where(listings['price'] != 0, np.log(listings['price']), 0)
#listings['price'] = np.log(listings.price)
roomcheck = listings[['price', 'room_type']].copy()
avg_prices = roomcheck.groupby('room_type')['price'].mean()
max_price = avg_prices.max()
score_list = []
for roomprice in avg_prices:
    score = (roomprice/max_price) * 100
    final_rounded_score = np.round(score, decimals = 2)
    score_list.append(final_rounded_score)  
#score_list=score_list[score_list['price'] != 0]
tomerge = pd.DataFrame({'room_type': ['Entire home/apt','Hotel room','Private room','Shared room'],
                        'room_type_scores':score_list})
final_listings = pd.merge(listings,tomerge,on ='room_type')


In [0]:
#part 2
test_df = final_table[['id','price','room_type_scores','accommodates','bathrooms','bedrooms','guests_included','host_total_listings_count','neighbour_scores']].copy()
test_df = test_df.dropna(how='any',axis=0)
#step I: extract neighbors and bathrooms and split the final-table as 0.2/0.8
train, val  = train_test_split(test_df, test_size=0.2, random_state=1)
# step II: build model on the tr data and show summary
model = smf.ols(formula = "price ~ id+ room_type_scores + accommodates +  bathrooms + bedrooms + guests_included + host_total_listings_count + neighbour_scores", data = train).fit()
model.summary()



<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.592
Model:                            OLS   Adj. R-squared:                  0.592
Method:                 Least Squares   F-statistic:                     1093.
Date:                Fri, 06 Dec 2019   Prob (F-statistic):               0.00
Time:                        01:19:11   Log-Likelihood:                -3634.0
No. Observations:                6029   AIC:                             7286.
Df Residuals:                    6020   BIC:                             7346.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
=============================================================================================
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                    -4.2002      0.162    -25.925      0.000      -4.518      -3.883
id                        -3.105e-09   5.39e-10     -5.757      0.000   -4.16e-09   -2.05e-09
room_type_scores              0.0309      0.001     33.327      0.000       0.029       0.033
accommodates                  0.0361      0.005      7.645      0.000       0.027       0.045
bathrooms                     0.0495      0.011      4.481      0.000       0.028       0.071
bedrooms                      0.1677      0.010     16.907      0.000       0.148       0.187
guests_included               0.0345      0.004      8.012      0.000       0.026       0.043
host_total_listings_count     0.0003    1.7e-05     19.083      0.000       0.000       0.000
neighbour_scores              0.0635      0.002     36.090      0.000       0.060       0.067
==============================================================================
Omnibus:                     1057.745   Durbin-Watson:                   1.942
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             7270.922
Skew:                           0.661   Prob(JB):                         0.00
Kurtosis:                       8.215   Cond. No.                     6.74e+08
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.74e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [0]:
# in the model, we have variables that we consider might have some effects on "price", 
# the intercept value shows us when all varaibles remain zero, the resulting price would be -4.2
# the variables coefficient values tell us: when all other variables keep same value:
# rising "id" by one, resulting price will rise by "-3.105e-09"
# rising "room_type_scores" by one, resulting price will rise by "0.0309"
# rising "accommodates" by one, resulting price will rise by 0.0361
#rising bathrooms number by one, resulting price will rise by 0.0495
# rising bedrooms number by one, resulting price will rise by 0.1677
# rise number of guest_included by one, resulting price will rise by 0.0345
# rising host_total_listings_count by one, resulting price will rise by 0.0003
# rising neighbour_scores by one, resulting price will rise by 0.0635
# we have an adjusted r^2 value of 0.592, which means after adjusting statistics 
# we have 59.2% of data that can be explained by this model
# we have 6029 observations in total, that's the number we get after cleaning the data.
# take alpha = 0.05, the p-value (P>|t|) tells us the effects of every variable included is considered
# as significant.


# step III: use te data to report rmse
predicted_validation = model.predict(val)
rmse = sqrt(mean_squared_error(te["price"], predicted_validation))
#final rmse is 
rmse

0.43666717142791706